In [1]:
import pandas as pd
import yfinance as yf
from pandas_datareader import data
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import talib 
from dateutil.relativedelta import relativedelta

from Tools import *
from Tools2 import *
import numpy as np

# Machine learning
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import cross_val_score
yf.pdr_override()

In [2]:
from itertools import combinations

lst = [6,7,8,10,11]
index=[]
i=0
while i<len(lst):
    for comb in combinations(lst,r=i):
        if comb==():
            continue
        index.append(list(comb))
    i+=1
print(index)
        


[[6], [7], [8], [10], [11], [6, 7], [6, 8], [6, 10], [6, 11], [7, 8], [7, 10], [7, 11], [8, 10], [8, 11], [10, 11], [6, 7, 8], [6, 7, 10], [6, 7, 11], [6, 8, 10], [6, 8, 11], [6, 10, 11], [7, 8, 10], [7, 8, 11], [7, 10, 11], [8, 10, 11], [6, 7, 8, 10], [6, 7, 8, 11], [6, 7, 10, 11], [6, 8, 10, 11], [7, 8, 10, 11]]


In [3]:
start="2020-12-31"
end="2022-12-31"
name="AAPL"

#MACD619RSI21_8020
stock_data=pct_change(name,start,end)
#stock_data=data.get_data_yahoo(tickers=name,start=start,end=end,progress=False)
    
stock_data=MACD_short(stock_data)
stock_data=RSI(stock_data)
stock_data=RSI_MACD(stock_data,21,80,20)


stock_data = stock_data.dropna()
del stock_data["real_situation"]
del stock_data["RSI6"]
del stock_data["RSI14"]

print(stock_data)
y = np.where(stock_data['Close'].shift(-1) > stock_data['Close'],1,-1)



                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2021-02-02  135.729996  136.309998  134.610001  134.990005  132.822083   
2021-02-03  135.759995  135.770004  133.610001  133.940002  131.788956   
2021-02-04  136.300003  137.399994  134.589996  137.389999  135.183533   
2021-02-05  137.350006  137.419998  135.860001  136.759995  134.764725   
2021-02-08  136.029999  136.960007  134.919998  136.910004  134.912552   
...                ...         ...         ...         ...         ...   
2022-12-23  130.919998  132.419998  129.639999  131.860001  131.299820   
2022-12-27  131.380005  131.410004  128.720001  130.029999  129.477585   
2022-12-28  129.669998  131.029999  125.870003  126.040001  125.504547   
2022-12-29  127.989998  130.479996  127.730003  129.610001  129.059372   
2022-12-30  128.410004  129.949997  127.430000  129.929993  129.378006   

              Volume  Close_price_per

In [4]:
high=0
count=0
for i in index:
    X = stock_data.iloc[:,i]
    split = int(0.7*len(stock_data))
    X_train, X_test, y_train, y_test = X[:split], X[split:], y[:split], y[split:]
    model = LogisticRegression()
    model = model.fit (X_train,y_train)
    probability = model.predict_proba(X_test)
    predicted = model.predict(X_test)
    
    #print(model.score(X_test,y_test))
    
    cross_val = cross_val_score(LogisticRegression(), X, y, scoring='accuracy', cv=10)
    mean_score=cross_val.mean()
    
    
    #print(mean_score)
    count+=1
    if mean_score>=high:
        high=mean_score
        print(high)
        print(i)
    

print(count)


0.503061224489796
[6]
0.519515306122449
[7]
0.519515306122449
[6, 7]
30
